# Título 

In [27]:
#importamos librerias al inicio
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

%matplotlib inline

In [30]:
path_data = "datos/sistema-unico-de-atencion-ciudadana-2020.csv"
columnas= ["contacto", "periodo","categoria","subcategoria",]

datos = pd.read_csv(path_data, usecols=columnas)

In [31]:
path = "datos/suaci_2020.csv"
datos1 = pd.read_csv(path)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 17, saw 2


In [4]:
datos

,contacto,periodo,categoria
0,00000001/20,202001,TRÁNSITO
1,00000002/20,202001,TRÁNSITO
2,00000003/20,202001,ARBOLADO Y ESPACIOS VERDES
3,00000004/20,202001,ARBOLADO Y ESPACIOS VERDES
4,00000005/20,202001,LIMPIEZA Y RECOLECCIÓN
...,...,...,...
574430,00564129/20,202012,"CONTROL EDILICIO, OBRAS Y CATASTRO"
574431,00593840/20,202012,"CONTROL EDILICIO, OBRAS Y CATASTRO"
574432,00554868/20,202012,ARBOLADO Y ESPACIOS VERDES
574433,00580083/20,202012,"CONTROL EDILICIO, OBRAS Y CATASTRO"


In [7]:
datos.periodo[-2:]

574433    202012
574434    202012
Name: periodo, dtype: int64

In [ ]:
datos["meses"]

In [6]:
datos.periodo.str[-2:]

AttributeError: Can only use .str accessor with string values!

In [8]:
datos['periodo'].str

AttributeError: Can only use .str accessor with string values!

In [9]:
datos.info

<bound method DataFrame.info of            contacto  periodo                           categoria
0       00000001/20   202001                            TRÁNSITO
1       00000002/20   202001                            TRÁNSITO
2       00000003/20   202001          ARBOLADO Y ESPACIOS VERDES
3       00000004/20   202001          ARBOLADO Y ESPACIOS VERDES
4       00000005/20   202001              LIMPIEZA Y RECOLECCIÓN
...             ...      ...                                 ...
574430  00564129/20   202012  CONTROL EDILICIO, OBRAS Y CATASTRO
574431  00593840/20   202012  CONTROL EDILICIO, OBRAS Y CATASTRO
574432  00554868/20   202012          ARBOLADO Y ESPACIOS VERDES
574433  00580083/20   202012  CONTROL EDILICIO, OBRAS Y CATASTRO
574434  00584748/20   202012              LIMPIEZA Y RECOLECCIÓN

[574435 rows x 3 columns]>

In [14]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574435 entries, 0 to 574434
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   contacto   574435 non-null  object
 1   periodo    574435 non-null  int64 
 2   categoria  390489 non-null  object
dtypes: int64(1), object(2)
memory usage: 13.1+ MB


In [16]:
str(datos.periodo)

'0         202001\n1         202001\n2         202001\n3         202001\n4         202001\n           ...  \n574430    202012\n574431    202012\n574432    202012\n574433    202012\n574434    202012\nName: periodo, Length: 574435, dtype: int64'

In [18]:
datos.periodo.astype(str).str[-2:]

0         01
1         01
2         01
3         01
4         01
          ..
574430    12
574431    12
574432    12
574433    12
574434    12
Name: periodo, Length: 574435, dtype: object

In [20]:
datos['mes'] = datos.periodo.astype(str).str[-2:]

In [22]:
datos.mes.unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)